In [ ]:
import pandas as pd

# Load RefFlat file
ref = pd.read_csv("refFlat", sep="\t", header=None)

# Set proper column names
ref.columns = [
    "gene_name", "transcript_name", "chrom", "strand",
    "tx_start_5prime", "tx_start_3prime",
    "cds_start", "cds_end",
    "exon_count", "exon_starts", "exon_ends"
]

# Extract strand-aware TSS
ref["tss"] = ref.apply(
    lambda row: row["tx_start_5prime"] if row["strand"] == "+" else row["tx_start_3prime"],
    axis=1
)

# Final TSS table for matching
tss_table = ref[["chrom", "tss"]].copy()


In [ ]:

ref.head()

,gene_name,transcript_name,chrom,strand,tx_start_5prime,tx_start_3prime,cds_start,cds_end,exon_count,exon_starts,exon_ends,tss
0,Wdsub1,NM_001159636,chr2,-,59855193,59882606,59855270,59878527,11,"59855193,59858609,59861560,59862619,59862816,5...","59855275,59858750,59861737,59862726,59862857,5...",59882606
1,Rbm18,NM_001159635,chr2,-,36116078,36136704,36117814,36134247,6,"36116078,36120812,36122851,36127214,36134134,3...","36117974,36120898,36122938,36127251,36134263,3...",36136704
2,Prrc2b,NM_001159634,chr2,+,32151147,32234537,32182511,32230742,32,"32151147,32182457,32183122,32185344,32187480,3...","32151291,32182626,32183300,32185447,32187553,3...",32151147
3,Ildr2,NM_001164528,chr1,+,166254138,166316832,166254375,166310795,10,"166254138,166269304,166270498,166291415,166294...","166254466,166269637,166270618,166291472,166294...",166254138
4,Perm1,NM_172417,chr4,+,156215926,156221307,156217000,156220222,4,"156215926,156216716,156219740,156220109,","156215975,156219185,156219866,156221307,",156215926


In [ ]:
# Extract strand-aware TSS
ref["tss"] = ref.apply(
    lambda row: row["tx_start_5prime"] if row["strand"] == "+" else row["tx_start_3prime"],
    axis=1
)

In [ ]:
# Final TSS table for matching
tss_table = ref[["chrom", "tss"]].copy()


In [10]:
tss_table.shape

(47382, 2)

In [ ]:

tss_table.isna().sum()

chrom    0
tss      0
dtype: int64

In [ ]:
peaks = pd.read_csv("ATAC-seq/refined_ATAC.csv")

# Define peak center
peaks["peak_center"] = peaks["Summit"]

# Keep minimal amnt of columns for distance task
peaks = peaks[["ImmGenATAC1219.peakID", "chrom", "peak_center"]].copy()


In [ ]:
#just to see if it worked 
peaks.head()


,ImmGenATAC1219.peakID,chrom,peak_center
0,ImmGenATAC1219.peak_1,chr1,3020786
1,ImmGenATAC1219.peak_2,chr1,3087226
2,ImmGenATAC1219.peak_3,chr1,3120109
3,ImmGenATAC1219.peak_4,chr1,3121485
4,ImmGenATAC1219.peak_5,chr1,3372787


In [28]:
peaks.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14865255 entries, 0 to 14865254
Data columns (total 3 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   ImmGenATAC1219.peakID  object
 1   chrom                  object
 2   peak_center            int64 
dtypes: int64(1), object(2)
memory usage: 340.2+ MB


In [36]:
tss_groups = tss_table.groupby("chrom")


In [37]:
def find_closest_tss(row):
    chrom = row["chrom"]
    center = row["peak_center"]
    
    if chrom not in tss_groups.groups:
        return np.nan
    
    tss_array = tss_groups.get_group(chrom)["tss"].values
    idx = (np.abs(tss_array - center)).argmin()
    return tss_array[idx]
